# IMDB Agent Prompt Testing Notebook

Use this notebook to test and refine prompts for the agent system.

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Question 1: When did The Matrix release?
result = df[df['Series_Title'].str.contains('Matrix', case=False, na=False)]
if not result.empty:
    year = result.iloc[0]['Released_Year']
    title = result.iloc[0]['Series_Title']
    print(f"Answer: {title} was released in {year}")
    result[['Series_Title', 'Released_Year']]
else:
    print("Movie not found")

## Question 2: Top 5 movies of 2019 by meta score

In [ ]:
# Question 2: Top 5 movies of 2019 by meta score
result = df[
    (df['Released_Year'] == 2019) & 
    (df['Meta_score'].notna())
].nlargest(5, 'Meta_score')[
    ['Series_Title', 'Released_Year', 'Meta_score', 'IMDB_Rating', 'Director']
].reset_index(drop=True)

print("Top 5 movies of 2019 by Meta Score:")
result

## Question 3: Top 7 comedy movies between 2010-2020 by IMDB rating

In [ ]:
# Question 3: Top 7 comedy movies between 2010-2020 by IMDB rating
result = df[
    (df['Released_Year'] >= 2010) & 
    (df['Released_Year'] <= 2020) &
    (df['Genre'].str.contains('Comedy', case=False, na=False)) &
    (df['IMDB_Rating'].notna())
].nlargest(7, 'IMDB_Rating')[
    ['Series_Title', 'Released_Year', 'Genre', 'IMDB_Rating', 'Meta_score', 'Director']
].reset_index(drop=True)

print("Top 7 comedy movies (2010-2020) by IMDB Rating:")
result

## Question 4: Top horror movies with meta score > 85 and IMDB rating > 8

In [ ]:
# Question 4: Top horror movies with meta score > 85 and IMDB rating > 8
result = df[
    (df['Genre'].str.contains('Horror', case=False, na=False)) &
    (df['Meta_score'] > 85) &
    (df['IMDB_Rating'] > 8)
].sort_values('IMDB_Rating', ascending=False)[
    ['Series_Title', 'Released_Year', 'Genre', 'IMDB_Rating', 'Meta_score', 'Director']
].reset_index(drop=True)

print(f"Found {len(result)} horror movies with Meta Score > 85 and IMDB Rating > 8:")
result

## Question 5: Top directors with movies grossing over $500M at least twice

In [ ]:
# Question 5: Top directors with movies grossing over $500M at least twice
# Filter movies with gross > 500M
high_grossing = df[
    (df['Gross'].notna()) & 
    (df['Gross'] > 500_000_000)
].copy()

# Group by director and count
director_counts = high_grossing.groupby('Director').agg({
    'Series_Title': 'count',
    'Gross': 'max',
    'IMDB_Rating': 'max'
}).rename(columns={'Series_Title': 'count_500m', 'Gross': 'highest_gross', 'IMDB_Rating': 'max_rating'})

# Filter directors with at least 2 movies > 500M
directors_with_2plus = director_counts[director_counts['count_500m'] >= 2].sort_values('highest_gross', ascending=False)

# Get the top movie for each director
result_list = []
for director in directors_with_2plus.index:
    director_movies = high_grossing[high_grossing['Director'] == director].sort_values('Gross', ascending=False)
    top_movie = director_movies.iloc[0]
    result_list.append({
        'Director': director,
        'Movies_Over_500M': int(director_counts.loc[director, 'count_500m']),
        'Highest_Grossing_Movie': top_movie['Series_Title'],
        'Highest_Gross': top_movie['Gross'],
        'Year': top_movie['Released_Year']
    })

result = pd.DataFrame(result_list)
print(f"Directors with at least 2 movies grossing over $500M:")
result

## Question 6: Top 10 movies with over 1M votes but lower gross earnings

In [ ]:
# Question 6: Top 10 movies with over 1M votes but lower gross earnings
# Filter movies with > 1M votes
high_votes = df[
    (df['No_of_Votes'].notna()) & 
    (df['No_of_Votes'] > 1_000_000) &
    (df['Gross'].notna())
].copy()

# Sort by gross (ascending = lower gross first), then by votes (descending)
result = high_votes.sort_values(['Gross', 'No_of_Votes'], ascending=[True, False]).head(10)[
    ['Series_Title', 'Released_Year', 'No_of_Votes', 'Gross', 'IMDB_Rating', 'Director']
].reset_index(drop=True)

print("Top 10 movies with >1M votes but lower gross earnings:")
result

## Question 7: Comedy movies with death or dead people in the plot (using Overview column)

In [ ]:
# Question 7: Comedy movies with death or dead people in the plot
# Filter comedy movies
comedy_movies = df[
    (df['Genre'].str.contains('Comedy', case=False, na=False)) &
    (df['Overview'].notna())
].copy()

# Search for death-related keywords in Overview
death_keywords = ['death', 'dead', 'dying', 'deceased', 'dies', 'died', 'killed', 'murder', 'suicide']
death_pattern = '|'.join(death_keywords)

result = comedy_movies[
    comedy_movies['Overview'].str.contains(death_pattern, case=False, na=False)
][['Series_Title', 'Released_Year', 'Genre', 'Overview', 'IMDB_Rating', 'Director']].reset_index(drop=True)

print(f"Found {len(result)} comedy movies with death themes:")
result

## Question 8: Summarize movie plots of Steven Spielberg's top-rated sci-fi movies

In [ ]:
# Question 8: Summarize movie plots of Steven Spielberg's top-rated sci-fi movies
# Filter Spielberg sci-fi movies
spielberg_scifi = df[
    (df['Director'] == 'Steven Spielberg') &
    (df['Genre'].str.contains('Sci-Fi', case=False, na=False)) &
    (df['IMDB_Rating'].notna())
].sort_values('IMDB_Rating', ascending=False)[
    ['Series_Title', 'Released_Year', 'IMDB_Rating', 'Meta_score', 'Overview']
].reset_index(drop=True)

print(f"Steven Spielberg's top-rated sci-fi movies ({len(spielberg_scifi)} found):")
print("\n" + "="*80)

for idx, row in spielberg_scifi.iterrows():
    print(f"\n{idx+1}. {row['Series_Title']} ({row['Released_Year']})")
    print(f"   IMDB Rating: {row['IMDB_Rating']}, Meta Score: {row['Meta_score']}")
    print(f"   Plot: {row['Overview']}")
    print("-"*80)

spielberg_scifi

## Question 9: Movies before 1990 with police involvement in the plot

Note: This requires semantic search (similarity search), not just keyword matching. 
For pandas, we'll do a keyword-based search, but note that the actual implementation 
should use vector similarity search (ChromaDB) for better results.

In [ ]:
# Question 9: Movies before 1990 with police involvement in the plot
# Filter movies before 1990
pre_1990 = df[
    (df['Released_Year'] < 1990) &
    (df['Released_Year'].notna()) &
    (df['Overview'].notna())
].copy()

# Search for police-related keywords (note: semantic search would be better)
police_keywords = ['police', 'cop', 'detective', 'officer', 'sheriff', 'law enforcement', 
                   'investigation', 'investigator', 'fbi', 'cia', 'interrogation', 'arrest']

police_pattern = '|'.join(police_keywords)

result = pre_1990[
    pre_1990['Overview'].str.contains(police_pattern, case=False, na=False)
][['Series_Title', 'Released_Year', 'Overview', 'IMDB_Rating', 'Genre', 'Director']].reset_index(drop=True)

print(f"Found {len(result)} movies before 1990 with police involvement (keyword-based search):")
print("Note: For better results, use semantic similarity search (ChromaDB) instead of keyword matching")
result

## Nice-to-Have Question 1: Al Pacino movies with >$50M gross and IMDB rating >= 8

This question requires checking if Al Pacino is in Star1, Star2, Star3, or Star4 columns.

## Nice-to-Have Question 2: Find similar movies based on Meta score and IMDB rating

This function finds movies with similar ratings to a given movie.

In [ ]:
# Nice-to-Have 2: Find similar movies based on Meta score and IMDB rating
def find_similar_movies(movie_title, n_similar=5, rating_tolerance=0.5, meta_tolerance=5):
    """
    Find movies with similar Meta score and IMDB rating to a given movie.
    
    Parameters:
    - movie_title: Name of the movie to find similarities for
    - n_similar: Number of similar movies to return
    - rating_tolerance: Tolerance for IMDB rating difference (default 0.5)
    - meta_tolerance: Tolerance for Meta score difference (default 5)
    """
    # Find the target movie
    target = df[df['Series_Title'].str.contains(movie_title, case=False, na=False)]
    
    if target.empty:
        return f"Movie '{movie_title}' not found"
    
    target_movie = target.iloc[0]
    target_rating = target_movie['IMDB_Rating']
    target_meta = target_movie['Meta_score']
    
    # Filter movies with similar ratings
    similar = df[
        (df['Series_Title'] != target_movie['Series_Title']) &  # Exclude the movie itself
        (df['IMDB_Rating'].notna()) &
        (df['Meta_score'].notna()) &
        (abs(df['IMDB_Rating'] - target_rating) <= rating_tolerance) &
        (abs(df['Meta_score'] - target_meta) <= meta_tolerance)
    ].copy()
    
    # Calculate similarity score (lower is better)
    similar['Rating_Diff'] = abs(similar['IMDB_Rating'] - target_rating)
    similar['Meta_Diff'] = abs(similar['Meta_score'] - target_meta)
    similar['Similarity_Score'] = similar['Rating_Diff'] + (similar['Meta_Diff'] / 10)
    
    result = similar.nsmallest(n_similar, 'Similarity_Score')[
        ['Series_Title', 'Released_Year', 'IMDB_Rating', 'Meta_score', 
         'Rating_Diff', 'Meta_Diff', 'Genre', 'Director']
    ].reset_index(drop=True)
    
    print(f"Movies similar to '{target_movie['Series_Title']}' ({target_movie['Released_Year']}):")
    print(f"Target: IMDB {target_rating}, Meta {target_meta}")
    print(f"\nFound {len(result)} similar movies:\n")
    
    return result

# Example: Find movies similar to "The Dark Knight"
similar_to_dark_knight = find_similar_movies("The Dark Knight", n_similar=5)
similar_to_dark_knight

In [1]:
import sys
sys.path.insert(0, '../src')

from agents import query_agent
from tools import get_duckdb_tool, get_chromadb_tool
from prompts import *
from config import *

## Test Query Agent

Run a complete end-to-end test:

In [2]:
# Test a query
query = "Top 5 movies of 2019 by meta score"
result = query_agent(query)

print(f"Query Type: {result['query_type']}")
print(f"\nResponse:\n{result['response']}")

if result.get('sql_query'):
    print(f"\nSQL Query:\n{result['sql_query']}")


Processing query: Top 5 movies of 2019 by meta score

[ROUTER] Classified as: STRUCTURED
[ROUTER] Reasoning: The query "Top 5 movies of 2019 by meta score" is classified as STRUCTURED. 

Reasoning: This query requires filtering the dataset to include only movies released in the year 2019 and then sorting tho...
[STRUCTURED ERROR] DuckDB database not found at /Users/faseehahmed26/Desktop/FaseehWorld/Interview Prep/realpage/realpage-imdb/notebooks/../data/imdb.duckdb. Please run 'python -m src.data_setup' first.

[SYNTHESIZER] Generated response (2218 chars)
Query Type: STRUCTURED

Response:
It looks like I couldn't access the database for the top movies of 2019 by meta score, but I can still share some widely recognized films from that year based on general knowledge and popular opinion!

Here are **5 notable movies from 2019** that received high acclaim:

1. **Parasite** 
   - **Genre:** Comedy, Drama, Thriller
   - **IMDB Rating:** 8.6
   - **Plot Summary:** A darkly comedic tale abo

## Test DuckDB Tool

Test SQL queries directly:

In [3]:
db_tool = get_duckdb_tool()

# Test a simple query
result = db_tool.execute_query("""
SELECT Series_Title, Released_Year, IMDB_Rating 
FROM imdb 
WHERE Released_Year = 2019 
ORDER BY IMDB_Rating DESC 
LIMIT 5
""")

print(result)

FileNotFoundError: DuckDB database not found at /Users/faseehahmed26/Desktop/FaseehWorld/Interview Prep/realpage/realpage-imdb/notebooks/../data/imdb.duckdb. Please run 'python -m src.data_setup' first.

In [ ]:
import pandas as pd